## Training

In [10]:
!rm -r checkpoints/voc_piou_l3*

### Smooth L1

In [ ]:
# first train with freezed backbone and lr=1e-3
!python train.py \
--snapshot imagenet \
--snapshot-path checkpoints/voc_l1 \
--loss l1 \
--phi 0 \
--gpu 0,1 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--freeze-backbone \
--batch-size 32 \
pascal /datasets/dataset/VOCdevkit/VOC2007

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'imagenet', 'freeze_backbone': True, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 0.001, 'batch_size': 32, 'phi': 0, 'gpu': '0,1', 'epochs': 50, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_l1', 'tensorboard_dir': 'logs/2021-02-20', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'loss': 'l1', 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2007'}
Epoch 1/50
157/157 [==============================] - ETA: 0s - loss: 1.3457 - classification_loss: 0.9490 - regression_loss: 0.3967
Epoch 00001: val_loss improved from inf to 1.65898, saving model to checkpoints/voc_l1/pascal.h5
157/157 [==============================] - 296s 2s/step - loss: 1.3457 - classification_loss: 0.9490 - regression_loss: 0.3967 - val_loss: 1.6590 - val_classification_loss: 1.1920 - val_regression_loss: 0.4670
Epoch

In [ ]:
# train all layers with lr=1e-4
!python train.py \
--snapshot checkpoints/voc_l1/pascal.h5 \
--snapshot-path checkpoints/voc_l1_finetuned \
--loss l1 \
--phi 0 \
--gpu 0,1 \
--epochs 100 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--lr 1e-4 \
--batch-size 16 \
pascal /datasets/dataset/VOCdevkit/VOC2007

### PIoU

In [ ]:
# first train with freezed backbone and lr=1e-3
!python train.py \
--snapshot imagenet \
--snapshot-path checkpoints/voc_piou_l3 \
--loss piou_l3 \
--phi 0 \
--gpu 0,1 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--freeze-backbone \
--batch-size 32 \
pascal /datasets/dataset/VOCdevkit/VOC2007

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'imagenet', 'freeze_backbone': True, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 0.001, 'batch_size': 32, 'phi': 0, 'gpu': '0,1', 'epochs': 50, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_piou_l3', 'tensorboard_dir': 'logs/2021-02-23', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'loss': 'piou_l3', 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2007'}


In [ ]:
# train all layers with lr=1e-4
!python train.py \
--snapshot checkpoints/voc_piou_l3/pascal.h5 \
--snapshot-path checkpoints/voc_piou_l3_finetuned \
--loss piou_l3 \
--phi 0 \
--gpu 0,1 \
--epochs 100 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--lr 1e-4 \
--batch-size 16 \
pascal /datasets/dataset/VOCdevkit/VOC2007

### GIoU

In [3]:
# first train with freezed backbone and lr=1e-3
!python train.py \
--snapshot imagenet \
--snapshot-path checkpoints/voc_giou \
--loss giou \
--phi 0 \
--gpu 0,1 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--freeze-backbone \
--batch-size 32 \
pascal /datasets/dataset/VOCdevkit/VOC2007

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'imagenet', 'freeze_backbone': True, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 0.001, 'batch_size': 32, 'phi': 0, 'gpu': '0,1', 'epochs': 50, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_giou', 'tensorboard_dir': 'logs/2021-02-22', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'loss': 'giou', 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2007'}
Epoch 1/50
157/157 [==============================] - ETA: 0s - loss: 1.3505 - classification_loss: 0.9436 - regression_loss: 0.4069
Epoch 00001: val_loss improved from inf to 1.63999, saving model to checkpoints/voc_giou/pascal.h5
157/157 [==============================] - 295s 2s/step - loss: 1.3505 - classification_loss: 0.9436 - regression_loss: 0.4069 - val_loss: 1.6400 - val_classification_loss: 1.1901 - val_regression_loss: 0.4499

In [3]:
# train all layers with lr=1e-4
!python train.py \
--snapshot checkpoints/voc_giou/pascal.h5 \
--snapshot-path checkpoints/voc_giou_finetuned \
--loss giou \
--phi 0 \
--gpu 0,1 \
--epochs 100 \
--no-evaluation \
--random-transform \
--compute-val-loss \
--lr 1e-4 \
--batch-size 16 \
pascal /datasets/dataset/VOCdevkit/VOC2007

{'dataset_type': 'pascal', 'detect_quadrangle': False, 'detect_text': False, 'snapshot': 'checkpoints/voc_giou_finetuned/pascal.h5', 'freeze_backbone': False, 'freeze_bn': False, 'weighted_bifpn': False, 'lr': 5.119999987073243e-05, 'batch_size': 16, 'phi': 0, 'gpu': '0,1', 'epochs': 100, 'steps': 10000, 'snapshot_path': 'checkpoints/voc_giou_finetuned', 'tensorboard_dir': 'logs/2021-02-23', 'snapshots': True, 'evaluation': False, 'random_transform': True, 'compute_val_loss': True, 'loss': 'giou', 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'pascal_path': '/datasets/dataset/VOCdevkit/VOC2007'}
Loading model, this may take a second...
Epoch 71/100
314/314 [==============================] - ETA: 0s - loss: 0.3015 - classification_loss: 0.1411 - regression_loss: 0.1604
Epoch 00071: val_loss improved from inf to 0.23520, saving model to checkpoints/voc_giou_finetuned/pascal.h5
314/314 [==============================] - 357s 1s/step - loss: 0.3015 - classification_loss: 0.

## Evaluation

### VOC 2007:
- Smooth l1: mAP = 0.7207
- PIoU:
    - mAP_smooth = 0.7135
    - mAP_L1 = 0.7186
    - mAP_L2 =
    - mAP_L3 = 
- GIoU: mAP = 0.7257

In [3]:
from eval.pascal import *
from generators.pascal import PascalVocGenerator
from model import efficientdet
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
phi = 0
weighted_bifpn = False
common_args = {
    'batch_size': 1,
    'phi': phi,
}
test_generator = PascalVocGenerator(
    '/datasets/dataset/VOCdevkit/VOC2007',
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
    **common_args
)

In [6]:
model_path = 'checkpoints/voc_piou_smooth_finetuned/pascal.h5'
input_shape = (test_generator.image_size, test_generator.image_size)
anchors = test_generator.anchors
num_classes = test_generator.num_classes()
model, prediction_model = efficientdet(phi=phi, num_classes=num_classes, weighted_bifpn=weighted_bifpn)
prediction_model.load_weights(model_path, by_name=True)

In [7]:
average_precisions = evaluate(test_generator, prediction_model, visualize=False)

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), test_generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP: {:.4f}'.format(mean_ap))

Running network: 100% (4952 of 4952) |###| Elapsed Time: 0:02:37 Time:  0:02:37
Parsing annotations: 100% (4952 of 4952) || Elapsed Time: 0:00:01 Time:  0:00:01


num_fp=256612.0, num_tp=10851.0
285 instances of class aeroplane with average precision: 0.7371
337 instances of class bicycle with average precision: 0.8289
459 instances of class bird with average precision: 0.6958
263 instances of class boat with average precision: 0.6096
469 instances of class bottle with average precision: 0.4827
213 instances of class bus with average precision: 0.7864
1201 instances of class car with average precision: 0.8143
358 instances of class cat with average precision: 0.8568
756 instances of class chair with average precision: 0.4620
244 instances of class cow with average precision: 0.7323
206 instances of class diningtable with average precision: 0.6333
489 instances of class dog with average precision: 0.8673
348 instances of class horse with average precision: 0.8516
325 instances of class motorbike with average precision: 0.8038
4528 instances of class person with average precision: 0.7726
480 instances of class pottedplant with average precision: 0